In [ ]:
pip install numpy scipy matplotlib pandas casadi pyfmi cvxpy control


In [ ]:
# Initialization of libraries
# Numerical computations and optimizations
import numpy as np
from scipy.optimize import minimize

# Plotting and visualization
import matplotlib.pyplot as plt

# Data manipulation
import pandas as pd

# Symbolic computations and MPC
import casadi as ca

# FMU interaction
from pyfmi import load_fmu

# Convex optimization (alternative or complementary to CasADi)
import cvxpy as cp

# Control system analysis (if needed)
import control as ctrl

import casadi

In [ ]:
# Define global simulation parameters
T = 3  # Time horizon length: horizo T = 5h
t_s = 1  # Time step size in hours or another time unit: ts = 1h -> for not losing too much on transitions for HESS, ideally ts should be smaller or equal to tau_CLD

# Define global parameters
# Medium parameters
GCV_H2 = 
NCV_H2 = 
M_H2 =
M_O2 =
M_H2O =
cp_H2 = 
cp_O2 =
cp_air =
cp_H2O = 
Delta_H0 =

In [ ]:
def MPC(ctrl_inputs, pv_slice, load_slice, k):
    
    # Initialize action dictionary
    action = {
        'P_battery': 0,
        'P_FC_sys': 0,
        'P_EL_sys': 0,
        'state_FC': 0,
        'state_EL': 0
    }

    # Use load_slice[0] and pv_slice[0] up to index [k-1]
    
    # Compute values for action['P_battery'], action['P_EL_sys'], action['P_FC_sys'], action['state_EL'] and action['state_FC']
    action = optimizer(ctrl_inputs, pv_slice, load_slice, k)
    
    # Convert action dictionary to a list of values
    action_list = list(action.values())
    print("Computed action list:", action_list)
    
    return action

In [ ]:
### i-th device (electrolyzer and fuel cell) cost function
def calculate_J_i(
    C_rep_i, N_H_i, C_OM_i,
    delta_ON_i, C_STB_OFF_i, delta_STB_OFF_i,
    C_OFF_STB_i, delta_OFF_STB_i, 
    C_OFF_ON_i, delta_OFF_ON_i,
    k, T
):
    J_i[k] = 0
    for j in range(T):
        on_term = (C_rep_i / N_H_i + C_OM_i) * delta_ON_i[k + j]
        stb_off_term = C_STB_OFF_i * sigma_STB_OFF_i[k + j]
        off_stb_term = C_OFF_STB_i * sigma_OFF_STB_i[k + j]
        on_off_term = C_OFF_ON_i * sigma_OFF_ON_i[k + j]
        
        J_i[k] += on_term + stb_off_term + off_stb_term + on_off_term
    
    return J_i[k]

# Example usage
# Delta arrays, assuming T = 5 for example
T = 5
# k defined by loop in MPC


# Define the parameters for the electrolyzer
C_rep_EL = 100  # Example replacement cost
N_H_EL = 50     # Example number of hours
C_OM_EL = 10    # Example O&M cost

C_STB_OFF_EL = 5  # Example standby-off cost
C_OFF_STB_EL = 3  # Example off-standby cost
C_ON_OFF_EL = 4   # Example off-on cost

# Defined by MPC
delta_ON_EL = [0] * T
sigma_OFF_STB_EL = [0] * T
sigma_STB_OFF_EL = [0] * T
sigma_ON_OFF_EL = [0] * T

# Calculate J_i(k) for k = 0 for the electorlyzer
J_EL[k] = calculate_J_i(
    C_rep_EL, N_H_EL, C_OM_EL, delta_ON_EL, 
    C_OFF_STB_EL, sigma_OFF_STB_EL,
    C_STB_OFF_EL, sigma_STB_OFF_EL,
    C_ON_OFF_EL, sigma_ON_OFF_EL,
    k=k, T=T
)

print(f"J_EL(k): {J_EL[k]}")




# Define the parameters for the fuel cell
C_rep_FC = 100  # Example replacement cost for the fuel cell
N_H_FC = 50     # Example number of hours for the fuel cell
C_OM_FC = 10    # Example O&M cost for the fuel cell

C_STB_OFF_FC = 5  # Example standby-off cost for the fuel cell
C_OFF_STB_FC = 3  # Example off-standby cost for the fuel cell
C_ON_OFF_FC = 4   # Example on-off cost for the fuel cell

# Defined by MPC or control logic
delta_ON_FC = [0] * T
sigma_OFF_STB_FC = [0] * T
sigma_STB_OFF_FC = [0] * T
sigma_ON_OFF_FC = [0] * T

# Calculate J_i(k) for k = 0 for the fuel cell
J_FC[k] = calculate_J_i(
    C_rep_FC, N_H_FC, C_OM_FC, delta_ON_FC,
    C_STB_OFF_FC, sigma_STB_OFF_FC,
    C_OFF_STB_FC, sigma_OFF_STB_FC,
    C_ON_OFF_FC, sigma_ON_OFF_FC,
    k=k, T=T
)

print(f"J_FC(k): {J_FC[k]}")


In [ ]:
# Battery cost function 
def calculate_J_battery(
    C_battery_inv, N_battery_cycles, P_battery_max,
    P_battery, k, T
):
    J_battery = 0
    for j in range(T):
        # Calculate the term inside the sum
        scaling_factor = C_battery_inv / (2 * N_battery_cycles * P_battery_max)
        # Sum up the absolute power term
        J_battery[k] += scaling_factor * abs(P_battery[k + j])
    
    return J_battery[k]

# Example usage
# Define the parameters
C_battery_inv = 1000  # Example investment cost for the battery
N_battery_cycles = 5000  # Example number of battery cycles
P_battery_max = 100  # Example maximum battery power

# Power profile (example data)
T = 5  # Time horizon length
P_battery   # Example power usage (could be charge/discharge)

# Calculate J_battery(k) for k = 0
J_battery[k] = calculate_J_battery(
    C_battery_inv, N_battery_cycles, P_battery_max,
    P_battery, k=k, T=T
)

print(f"J_battery(k): {J_battery[k]}")


In [ ]:
# Grid cost function 
def calculate_J_grid(
    Gamma_sale, P_grid_sale, Gamma_purchase, P_grid_purchase,
    t_s, k, T
):
    J_grid[k] = 0
    for j in range(T):
        # Calculate the cost contribution for this time step
        sale_cost = -Gamma_sale[k + j] * P_grid_sale[k + j]
        purchase_cost = Gamma_purchase[k + j] * P_grid_purchase[k + j]
        # Sum the costs, considering the time step size
        J_grid[k] += (sale_cost + purchase_cost) * t_s
    
    return J_grid[k]

# Example usage

# Define parameters: price profiles read from .csv or other source
Gamma_sale   # Sale price per unit of power
Gamma_purchase   # Purchase price per unit of power


P_grid_sale  # Power sold to the grid at each time step
P_grid_purchase   # Power purchased from the grid at each time step

# Calculate J_grid(k) for k = 0
J_grid[k] = calculate_J_grid(
    Gamma_sale, P_grid_sale, Gamma_purchase, P_grid_purchase,
    t_s, k=k, T=T
)

print(f"J_grid(0): {J_grid[k]}")


In [ ]:
# Total cost function 
J[k] = J_FC[k] + J_EL[k] + J_battery[k] + J_grid[k]

In [ ]:
# Grid constraints
P_grid[k] = P_grid_purchase[k] - P_grid_sale[k]

# Define parameters
P_grid_max = 5000


grid_constraints = [
    P_grid_sale[k] >= 0,
    P_grid_purchase[k] >= 0,
    P_grid_sale[k] <= P_grid_max,
    P_grid_purchase[k] <= P_grid_max
]


In [ ]:
# Battery constraints

# Define parameters
P_battery_nom = 5000
E_battery_nom = 5000
eta_battery_ch = 0.98
eta_battery_disch = 0.98
V_nom = 230
C_nom = 5000
SOC_battery_min = 0.1
SOC_battery_max = 0.9
SOE_battery_min = 0.2 * E_battery_nom
SOE_battery_max = 0.9 * E_battery_nom

# Initialization of problem variables - pass to FMU as well
SOC_battery_init 
SOE_battery_init 


# Power
P_battery[k] = P_battery_ch[k] - P_battery_disch[k]
P_battery[k] >= - 0.9 * P_battery_nom
P_battery[k] <= 0.9 * P_battery_nom


# SOC
# Define initial state of charge
SOC_battery[0] = SOC_battery_init
# SOC constraints
SOC_battery[k] = SOC_battery[k-1] + (P_battery_ch[k] * np.sqrt(eta_battery_ch) + P_battery_disch[k] / p.sqrt(eta_battery_disch)) * t_s / (V_nom * C_nom)
SOC_battery[k] >= SOC_battery_min
SOC_battery[k] <= SOC_battery_max


# SOE
# Define initial state of energy
SOE_battery[0] = SOE_battery_init
# SOE constraints
SOE_battery[k] = SOE_battery[k-1] + P_battery[k] * t_s
SOE_battery[k] >= SOE_battery_min
SOE_battery[k] <= SOE_battery_max


In [ ]:
# Hydrogen storage constraints

# Define parameters
P_hess_nom = 5000 # W
E_hess_nom = 402 # Ws or J
eta_EL = 0.6
eta_FC = 0.4
n_H2,max = 21510 #mol/s
SOC_hess_min = 0.1
SOC_hess_max = 0.99
SOE_hess_min = 0.2 * E_hess_nom # Ws or J
SOE_hess_max = 0.9 * E_hess_nom # Ws or J

# Initialization of problem variables - pass to FMU as well
SOC_hess_init 
SOE_hess_init 

# Power
#P_hess[k] = P_EL[k] - P_FC[k]


# SOC
# Define initial state of charge
SOC_hess[0] = SOC_hess_init
# SOC constraints
SOC_hess[k] = SOC_hess[k-1] + (alpha * z_EL_in[k] - beta * z_FC_out[k]) * t_s / n_H2,max 
alpha = eta_EL / ( GCV_H2 *M_H2)
beta = eta_FC * lambda_H2 / ( NCV_H2 *M_H2)
SOC_hess[k] >= SOC_hess_min
SOC_hess[k] <= SOC_hess_max


# SOE
# Define initial state of energy
SOE_hess[0] = SOE_hess_init
# SOE constraints
SOE_hess[k] = SOE_hess[k-1] + (eta_EL * z_EL_in[k] -  z_FC_out[k]/eta_FC) * t_s
SOE_hess[k] >= SOE_hess_min
SOE_hess[k] <= SOE_hess_max


In [ ]:
# Discrete logical states 1

# Define parameters
M = 10000
m = 1
P_EL_STB = 285.7
P_EL_max = 5000
P_EL_min = 0.1 * P_EL_max
P_FC_STB = 300
P_FC_max = 4300
P_FC_min = 300

# Binary decision variables for EL
delta_ON_EL 
delta_OFF_EL
delta_STB_EL

z_min_pos_EL
z_max_pos_EL
z_stb_pos_EL 
z_stb_neg_EL 
z_pos_EL 
z_neg_EL 

# Binary decision variables for FC
delta_ON_FC 
delta_OFF_FC
delta_STB_FC

z_min_pos_FC
z_max_neg_FC
z_stb_pos_FC 
z_stb_neg_FC 
z_pos_FC 
z_neg_FC


def enforce_z_constraints(P, M, k, z_pos, z_neg, P_stb, z_stb_pos, z_stb_neg, P_min, z_min_pos, P_max, z_max_neg):
    # Constraints for P_i(k)
    constraints = []
    
    # 1. P_i(k) <= M * z^>=0_i(k)
    constraints.append(P[k] <= M * z_pos[k])
    
    # 2. -P_i(k) <= M * (1 - z^>=0_i(k))
    constraints.append(-P[k] <= M * (1 - z_pos[k]))
    
    # 3. -P_i(k) <= M * z^<=0_i(k)
    constraints.append(-P[k] <= M * z_neg[k])
    
    # 4. P_i(k) <= M * (1 - z^<=0_i(k))
    constraints.append(P[k] <= M * (1 - z_neg[k]))
    
    # 5. P_i(k) + P^STB_i <= M * z^>=P^STB_i(k)
    constraints.append(P[k] + P_stb <= M * z_stb_pos[k])
    
    # 6. -P_i(k) + P^STB_i <= M * (1 - z^>=P^STB_i(k))
    constraints.append(-P[k] + P_stb <= M * (1 - z_stb_pos[k]))
    
    # 7. -P_i(k) + P^STB_i <= M * z^<=P^STB_i(k)
    constraints.append(-P[k] + P_stb <= M * z_stb_neg[k])
    
    # 8. P_i(k) + P^STB_i <= M * (1 - z^<=P^STB_i(k))
    constraints.append(P[k] + P_stb <= M * (1 - z_stb_neg[k]))
    
    # 9. P_i(k) >= P^min_i * z^>=P^min_i(k)
    constraints.append(P[k] >= P_min * z_min_pos)
    
    # 10. -P_i(k) + P^min_i <= M * (1 - z^>=P^min_i(k))
    constraints.append(-P[k] + P_min <= M * (1 - z_min_pos[k]))
    
    # 11. -P_i(k) + P^max_i <= M * z^<=P^max_i(k)
    constraints.append(-P[k] + P_max <= M * z_max_neg[k])
    
    # 12. P_i(k) >= P^max_i * (1 - z^<=P^max_i(k))
    constraints.append(P[k] >= P_max * (1 - z_max_neg[k]))
    
    return constraints



# Check constraints for EL
z_constraints_EL = enforce_z_constraints(
    P_EL, M, z_pos_EL, z_neg_EL, P_EL_STB, z_stb_pos_EL, z_stb_neg_EL,
    P_EL_min, z_min_pos_EL, P_EL_max, z_max_neg_EL
)

# Check constraints for FC
z_constraints_FC = enforce_z_constraints(
    P_FC, M, z_pos_FC, z_neg_FC, P_FC_STB, z_stb_pos_FC, z_stb_neg_FC,
    P_FC_min, z_min_pos_FC, P_FC_max, z_max_neg_FC
)

# Output results
print("Constraints for EL:", z_constraints_EL)
print("Constraints for FC:", z_constraints_FC)



In [ ]:
# Discrete logical states 2

def enforce_state_constraints(k, delta_ON, delta_OFF, delta_STB, z_min_pos, z_max_neg, z_stb_pos, z_stb_neg, z_off_pos, z_off_neg):
    constraints = []

    # 1. (1 - δ^ON_i(k)) + z^>=P^min_i(k) >= 1
    constraints.append((1 - delta_ON) + z_min_pos >= 1)

    # 2. (1 - δ^ON_i(k)) + z^<=P^max_i(k) >= 1
    constraints.append((1 - delta_ON) + z_max_neg >= 1)

    # 3. (1 - δ^STB_i(k)) + z^>=P^STB_i(k) >= 1
    constraints.append((1 - delta_STB) + z_stb_pos >= 1)  

    # 4. (1 - δ^STB_i(k)) + z^<=P^STB_i(k) >= 1
    constraints.append((1 - delta_STB) + z_stb_neg >= 1)  

    # 5. (1 - δ^OFF_i(k)) + z^>=0_i(k) >= 1
    constraints.append((1 - delta_OFF) + z_off_pos >= 1)  

    # 6. (1 - δ^OFF_i(k)) + z^<=0_i(k) >= 1
    constraints.append((1 - delta_OFF) + z_off_neg >= 1) 

    return constraints


# Check/Write? constraints for EL
states_constraints_EL = enforce_state_constraints(
    delta_ON_EL[k], z_min_pos_EL[k], z_max_neg_EL[k], z_stb_pos_EL[k], z_stb_neg_EL[k], z_pos_EL[k], z_neg_EL[k]
)

# Check/Write? constraints for FC
states_constraints_FC = enforce_state_constraints(
    delta_ON_FC[k], z_min_pos_FC[k], z_max_neg_FC[k], z_stb_pos_FC[k], z_stb_neg_FC[k], z_pos_FC[k], z_neg_FC[k]
)

# Output results
print("Constraints for EL:", states_constraints_EL)
print("Constraints for FC:", states_constraints_FC)


In [ ]:
# Discrete logical states 3

# each device can only be in 1 state at a time
delta_ON_EL[k] + delta_OFF_EL[k] + delta_STB_EL[k] = 1 
delta_ON_FC[k] + delta_OFF_FC[k] + delta_STB_FC[k] = 1

# electrolyzer and fuel cell cannot be ON at the same time
delta_ON_EL[k] + delta_ON_FC[k] = 1 #to verify


In [ ]:
# States transitions 1

# Binary decision variables for EL
delta_ON_EL 
delta_OFF_EL
delta_STB_EL

sigma_ON_OFF_EL
sigma_OFF_ON_EL
sigma_STB_ON_EL 
sigma_ON_STB_EL 
sigma_STB_OFF_EL 
sigma_OFF_STB_EL 


# Binary decision variables for FC
delta_ON_FC 
delta_OFF_FC
delta_STB_FC

sigma_ON_OFF_FC
sigma_OFF_ON_FC
sigma_STB_ON_FC 
sigma_ON_STB_FC 
sigma_STB_OFF_FC 
sigma_OFF_STB_FC



def enforce_transition_constraints(
    delta_ON_prev, delta_ON, delta_OFF_prev, delta_OFF, delta_STB_prev, delta_STB,
    sigma_ON_OFF, sigma_OFF_ON, sigma_STB_ON, sigma_ON_STB, sigma_STB_OFF, sigma_OFF_STB
):
    constraints = []

    # Constraints ON to OFF
    constraints.append(-delta_ON_prev + sigma_ON_OFF <= 0)
    constraints.append(-delta_OFF + sigma_ON_OFF <= 0)
    constraints.append(delta_ON_prev + delta_OFF - sigma_ON_OFF <= 1)

    # Constraints OFF TO ON
    constraints.append(-delta_OFF_prev + sigma_OFF_ON <= 0)
    constraints.append(-delta_ON + sigma_OFF_ON <= 0)
    constraints.append(delta_OFF_prev + delta_ON - sigma_OFF_ON <= 1)

    # Constraints STB TO ON
    constraints.append(-delta_STB_prev + sigma_STB_ON <= 0)
    constraints.append(-delta_ON + sigma_STB_ON <= 0)
    constraints.append(delta_STB_prev + delta_ON - sigma_STB_ON <= 1)
    
    # Constraints ON TO STB
    constraints.append(-delta_ON_prev + sigma_ON_STB <= 0)
    constraints.append(-delta_STB + sigma_ON_STB <= 0)
    constraints.append(delta_ON_prev + delta_STB - sigma_ON_STB <= 1)
    
    # Constraints STB TO OFF
    constraints.append(-delta_STB_prev + sigma_STB_OFF <= 0)
    constraints.append(-delta_OFF + sigma_STB_OFF <= 0)
    constraints.append(delta_STB_prev + delta_OFF - sigma_STB_OFF <= 1)

    # Constraints OFF TO STB
    constraints.append(-delta_OFF_prev + sigma_OFF_STB <= 0)
    constraints.append(-delta_STB + sigma_OFF_STB <= 0)
    constraints.append(delta_OFF_prev + delta_STB - sigma_OFF_STB <= 1)

    return constraints


# Check constraints for EL
constraints_EL = enforce_transition_constraints(
    delta_ON_EL[k-1], delta_ON_EL[k],  delta_OFF_EL[k-1],  delta_OFF_EL[k], delta_STB_EL[k-1], delta_STB_EL[k],
    sigma_ON_OFF_EL[k], sigma_OFF_ON_EL[k] = 0, sigma_STB_ON_EL[k], sigma_ON_STB_EL[k], sigma_STB_OFF_EL[k], sigma_OFF_STB_EL[k]
)

# Check constraints for FC
constraints_FC = enforce_transition_constraints(
    delta_ON_FC[k-1], delta_ON_FC[k],  delta_OFF_FC[k-1],  delta_OFF_FC[k], delta_STB_FC[k-1], delta_STB_FC[k],
    sigma_ON_OFF_FC[k], sigma_OFF_ON_FC[k] = 0, sigma_STB_ON_FC[k], sigma_ON_STB_FC[k], sigma_STB_OFF_FC[k], sigma_OFF_STB_FC[k]
)

sigma_OFF_ON_EL = np.zeros(T)
sigma_OFF_ON_FC = np.zeros(T)

# Output results
print("Constraints for EL:", constraints_EL)
print("Constraints for FC:", constraints_FC)


In [ ]:
# States transitions 2
# Initialize parameters
T_CLD_EL = 1
T_CLD_FC = 1


def enforce_startup_constraints(sigma_STB_OFF, T_CLD, k):
    # Initialize a list to store constraint satisfaction statuses
    constraints = []

    # Generate tau_CLD as a vector of time steps: tau_CLD = (k+1, k+2, ..., k+T_CLD)
    tau_CLD = np.arange(k + 1, k + T_CLD)

    # First constraint: σ^STB_OFF_i(k) - σ^STB_OFF_i(k-1) <= σ^STB_OFF_i(τ_CLD)
    for tau in tau_CLD:
       # if tau < len(sigma_STB_OFF):
        constraints.append(sigma_STB_OFF[k] - sigma_STB_OFF[k-1] <= sigma_STB_OFF[tau])
    
    # Second constraint: Sum of σ^STB_OFF_i(k-T_CLD) to σ^STB_OFF_i(k) <= T_CLD
    if k >= T_CLD:
        sigma_sum = sum(sigma_STB_OFF[k - T_CLD:k])  # Sum over the period
        constraints.append(sigma_sum <= T_CLD)
    else:
        sigma_sum = sum(sigma_STB_OFF[0:k])  # Sum over the period
        constraints.append(sigma_sum <= T_CLD)
        
    return constraints



# Electrolyzer start-up time
startup_constraints_EL = enforce_startup_constraints(sigma_STB_OFF_EL, T_CLD_EL, k)


# Fuel cell start-up time
startup_constraints_FC = enforce_startup_constraints(sigma_STB_OFF_FC, T_CLD_FC, k)


In [ ]:
# Electrolyzer dynamics

# Initialize parameters
GCV_H2 = 
M_H2 =
M_O2 =
M_H2O =
cp_H2 = 
cp_O2 = 
eta_EL = 0.6
eta_F_EL = 
R = 8.314
F = 
N_EL = 20
V_tn = 1.482
R_th = 
T_H2_nom = 273.15 + 40
T_EL_nom = 273.15 + 50
T_amb = 273.15 + 23.5
e_dry_spec = 1400 * 3600 * t_s? # Ws/kg
COP_cw = 4
K=1.4
eta_comp = 0.72
p_max = 350*10**5 # 350 bar = 350 * 10^5 Pa
p_comp_in = 1.1325 *10**5 # 1 atm = 1.1325 bar = 1 * 10^5 Pa
P_EL_STB = 
h_H2 = cp_H2 * (T_H2_nom - T_amb)
h_O2 = cp_O2 * (T_H2_nom - T_amb)
P_pump_disch =
P_pump_circ = 
Q_loss_EL = (T_EL,nom - T_amb)/R_th



# mass 
dot_m_H2_EL[k] = eta_EL * P_EL[k] / GCV_H2
dot_n_H2_EL[k] = dot_m_H2_EL[k] / M_H2
dot_n_O2_EL[k] = 0.5 * dot_n_H2_EL[k] 

# current
I_EL[k] = 2 * F * dot_m_H2_EL[k] / (M_H2 * N_EL * eta_F_EL)

# cooling & temperature
Q_cooling_EL[k] = P_gen[k] + P_pump,disch - H_flow_prod[k] - Q_loss_EL[k]
P_gen[k] = P_EL[k] - N_EL * V_tn * I_EL[k]
H_flow_prod[k] = dot_n_H2_EL[k] * h_H2 + dot_n_O2_EL[k] * h_O2 

# auxiliaries power
P_EL_aux[k] = P_dry[k] + P_pump,circ + P_cooling_EL[k] + P_comp,H2[k]
P_dry[k] = e_dry,spec * dot_m_H2_EL[k] 
P_cooling_EL[k] = Q_cooling_EL[k]/COP_cw
RECAST: P_comp,H2[k] = dot_n_H2_EL[k] / eta_comp * (K/K-1) * R * T_H2,nom * ( (p_max * SOC_hess[k]/p_comp_in)**(K-1/K) - 1 )


# electrolyzer system power
P_EL_sys[k] = z_EL_in[k] + z_EL_aux[k] + P_EL_STB * delta_STB_EL[k]


In [ ]:
# Fuel cell dynamics

# Initialize parameters
NCV_H2 = 
M_H2 =
M_O2 =
M_H2O =
cp_H2 = 
cp_air =
cp_H2O = 
Delta_H0 =
Delta_HT = cp_H2O * (T_FC_nom - T_amb)
eta_FC = 0.4
eta_F_FC = 
R = 8.314
F = 
N_FC = 36
V_tn = 1.482
R_th_FC = 
T_Air_in = 273.15 + 23.5
T_H2_in = 273.15 + 40
T_FC,nom = 273.15 + 72
T_amb = 273.15 + 23.5
lambda_H2 = 1.5
lambda_O2 = 2.05
COP_cw = 4
K=1.4
eta_comp_air = 0.72
p_comp_in = 1.1325 *10**5 # 1 atm = 1.1325 bar = 1 * 10^5 Pa
p_comp_out = 3 *10**5 # pressure at cathode = 3 bar
P_FC_STB = 
h_H2_in = cp_H2 * (T_H2,nom - T_amb) # Delta T or just T_in?
h_H2_out = cp_H2 * (T_FC,nom - T_amb)
h_air_in = cp_air * (T_amb - T_amb) 
h_air_out = cp_air * (T_FC,nom - T_amb)
Q_loss_FC = (T_FC,nom - T_amb)/R_th



# mass 
dot_m_H2_react[k] = eta_FC * P_FC[k] / NCV_H2
dot_m_H2_in[k] = dot_m_H2_react[k] * lambda_H2
dot_m_H2_out[k] = dot_m_H2_in[k] - dot_m_H2_react[k]
dot_n_H2_react[k] = dot_m_H2_react[k] / M_H2

dot_n_O2_react[k] = 0.5 * dot_n_H2_react[k] 
dot_n_air_react[k] = dot_n_O2_react[k] / 0.21
dot_m_O2_react[k] = dot_n_O2_react[k] * M_O2
dot_m_O2_in[k] = dot_m_O2_react[k] * lambda_O2
dot_m_air_in[k] = dot_m_O2_in[k] / 0.21
dot_m_O2_out[k] = dot_m_O2_in[k] - dot_m_O2_react[k]
dot_m_air_out[k] = dot_m_O2_out[k]/0.21

dot_m_H2O_gen[k] = dot_n_H2_react[k] * M_H2O

# current
I_FC[k] = 2 * F * eta_F_FC * dot_m_H2_react[k] / (M_H2 * N_FC)

# cooling & temperature
Q_cooling_FC[k] = Q_tot[k] +  Q_sens[k] - P_FC[k] - Q_loss_FC
Q_tot[k] = dot_m_H2_react[k] * Delta_H0
Q_sens[k] = dot_m_H2_in[k] * h_H2_in -  dot_m_H2_out[k] * h_H2_out + dot_m_air_in[k] * h_air_in -  dot_m_air_out[k] * h_air_out 
                - dot_m_H2O_gen[k] * Delta_HT/M_H2O

# auxiliaries power
P_FC_aux[k] = P_cooling_FC[k] + P_comp_air[k] 
P_cooling[k] = Q_cooling_FC[k]/COP_cw
P_comp_air[k] = dot_n_air_EL[k] / eta_comp_air * (K/K-1) * R * T_Air_in * ( (p_comp_out/p_comp_in)**(K-1/K) - 1 )


# electrolyzer system power
P_FC_sys[k] = z_FC_out[k] - z_FC_aux[k] - P_FC_STB * delta_STB_FC[k]



In [ ]:
# Electrolyzer and fuel cell operating constraints

# Initialize parameters
P_EL_min = 0.1 * P_EL_max
P_EL_max = 5000
P_FC_min = 300
P_FC_max = 4300
r_EL = 20 * t_s # 20 W/s 
r_FC = 5 * t_s # 5 W/s 


# Min and max power boundaries
P_EL[k] >= P_EL_min
P_EL[k] <= P_EL_max

P_FC[k] >= P_FC_min
P_FC[k] <= P_FC_max


# Ramping constraints
#np.abs((P_EL[k+1]-P_EL[k])*delta_ON_EL[k]) <= r_EL
#np.abs((P_FC[k+1]-P_FC[k])*delta_ON_FC[k]) <= r_FC
np.abs(z_EL_in[k+1]-z_EL_in[k]) <= r_EL
np.abs(z_FC_out[k+1]-z_FC_out[k]) <= r_EL


# Logical powers: recast P_EL(k) * delta_ON_EL(k) and P_FC(k) * delta_ON_FC(k) and replace in the code
z_EL_in[k] >= m * delta_ON_EL[k]
z_EL_in[k] <= M * delta_ON_EL[k]
z_EL_in[k] >= P_EL[k] - M * (1 - delta_ON_EL[k])
z_EL_in[k] <= P_EL[k] + m * (1 - delta_ON_EL[k])


z_FC_out[k] >= m * delta_FC_EL[k]
z_FC_out[k] <= M * delta_FC_EL[k]
z_FC_out[k] >= P_FC[k] - M * (1 - delta_ON_FC[k])
z_FC_out[k] <= P_FC[k] + m * (1 - delta_ON_FC[k])

# Logical powers for auxiliaries: recast P_EL,aux(k) * delta_ON_EL(k) and P_FC,aux(k) * delta_ON_FC(k) and replace in the code
z_EL_aux[k] >= m_aux * delta_ON_EL[k]
z_EL_aux[k] <= M_aux * delta_ON_EL[k]
z_EL_aux[k] >= P_EL[k] - M_aux * (1 - delta_ON_EL[k])
z_EL_aux[k] <= P_EL[k] + m_aux * (1 - delta_ON_EL[k])


z_FC_aux[k] >= m_aux * delta_FC_EL[k]
z_FC_aux[k] <= M_aux * delta_FC_EL[k]
z_FC_aux[k] >= P_FC[k] - M_aux * (1 - delta_ON_FC[k])
z_FC_aux[k] <= P_FC[k] + m_aux * (1 - delta_ON_FC[k])


In [ ]:
# Power balancing equation
P_PV + P_grid + P_battery_disch + P_FC_sys  = P_load + P_battery_ch + P_EL_sys

In [ ]:
def optimizer_V1(T, t_s, c_el, pv, load, SOE_initial, LOH_initial): 
    # adapt inputs: T (horizon of simulation), ts (sampling time), SOE_battery, LOH_hess, pv, load, c_el (electricity prices)
    # states: SOE_battery, LOH_hess
    # inputs to FMU: P_battery, P_FC_sys (start with P_FC), P_El_sys (start with P_El), state_FC, state_EL

    # ITERATION 1: no auxiliaries, no thermal model, no transitions
    # inputs variables in MPC: P_battery_disch, P_battery_ch, P_grid, P_FC, P_FC_sys, P_FC_aux, delta_ON_FC, delta_OFF_FC, delta_STB_FC, P_EL, P_EL_sys, P_EL_aux, delta_ON_EL, delta_OFF_EL, delta_STB_EL
    
    
    # Access global parameters
    global P_grid_max, P_battery_nom, eta_batt_ch, eta_batt_disch, SOE_min, SOE_max
    global LOH_min, LOH_max, C_max, eta_El, eta_FC, P_EL_min, P_EL_max, P_EL_STB, P_FC_min, P_FC_max, P_FC_STB, r_EL, r_FC
    global alpha_EL, alpha_FC, c_battery

    N = int(T // t_s)
    
    # Initialize
    import casadi as ca
    opti = ca.Opti()

    # -----------------------------
    # Variables and solver
    # -----------------------------

    SOE = opti.variable(1,N+1)  # state
    LOH = opti.variable(1,N+1)  # state
    P_batt_disch = opti.variable(1,N)       # input
    P_batt_ch = opti.variable(1,N)   # input
    P_grid = opti.variable(1,N) # input 
    #P_FC_sys = opti.variable(1,N) # input
    #P_FC_aux = opti.variable(1,N) # input
    P_FC = opti.variable(1,N) # input
    delta_ON_FC = opti.variable(1,N) # input
    #delta_STB_FC = opti.variable(1,N) # input
    delta_OFF_FC = opti.variable(1,N) # input
    #P_EL_sys = opti.variable(1,N) # input
    #P_EL_aux = opti.variable(1,N) # input
    P_EL = opti.variable(1,N) # input
    delta_ON_EL = opti.variable(1,N) # input
    #delta_STB_EL = opti.variable(1,N) # input
    delta_OFF_EL = opti.variable(1,N) # input

    # delta_HP is a discrete variable (binary)
    discrete_var = [0]*(N+1) + [0]*(N+1) + [0]*N + [0]*N + [0]*N + [0]*N + [1]*N + [1]*N + [0]*N + [1]*N +  [1]*N 

    # Solver
    opti.solver('bonmin', {'discrete': discrete_var, 'bonmin.tol': 1e-4, 'bonmin.print_level': 0, 'print_time': 0})

    # -----------------------------
    # Constraints
    # -----------------------------

    # Initial storage level
    opti.subject_to(SOE[0] == SOE_initial)
    opti.subject_to(LOH[0] == LOH_initial)

    # Constraints at every time step
    for t in range(N):

            # Bounds on grid
            opti.subject_to(pv[t] + P_grid[t] + P_batt_disch[t] + P_FC[t] == load[t] + P_batt_ch[t] + P_EL[t])
            opti.subject_to(P_grid[t]>=0)
            opti.subject_to(P_grid[t]<=P_grid_max)

            # Bounds on battery and SOE
            opti.subject_to(P_batt_disch[t]>=0)
            opti.subject_to(P_batt_disch[t]<=0.9 * P_battery_nom)
            opti.subject_to(P_batt_ch[t]>=0)
            opti.subject_to(P_batt_ch[t]<=0.9 * P_battery_nom)
            
            opti.subject_to(SOE[t+1] == SOE[t] + (P_batt_ch[t] * eta_batt_ch - P_batt_disch[t]/eta_batt_disch) * t_s)
            opti.subject_to(SOE[t] >= SOE_min)
            opti.subject_to(SOE[t] <= SOE_max)
            
            # Bounds on hydrogen storage 
            opti.subject_to(LOH[t+1] == LOH[t] + (P_EL[t] * eta_EL - P_FC[t]/eta_FC) * t_s /C_max)
            opti.subject_to(LOH[t] >= LOH_min)
            opti.subject_to(LOH[t] <= LOH_max)                

            # Constraints on electrolyzer
            # Bounds when ON                
            opti.subject_to(P_EL[t] >= delta_ON_EL[t] * P_EL_min)
            opti.subject_to(P_EL[t] <= delta_ON_EL[t] * P_EL_max)
            # Bounds when STB
            #opti.subject_to(P_EL_aux[t] == delta_STB_EL[t] * P_EL_STB)
            # Ramping constraints                
            if t < N-1:                
                opti.subject_to(P_EL[t+1] - P_EL[t] <= r_EL)
                opti.subject_to(P_EL[t] - P_EL[t+1] <= r_EL)
                            
            # Constraints on fuel cell
            # Bounds when ON
            opti.subject_to(P_FC[t] >= delta_ON_FC[t] * P_FC_min)
            opti.subject_to(P_FC[t] <= delta_ON_FC[t] * P_FC_max)
            # Bounds when STB
            #opti.subject_to(P_FC_aux[t] == delta_STB_FC[t] * P_FC_STB)
            # Ramping constraints                
            if t < N-1:                
                opti.subject_to(P_FC[t+1] - P_FC[t] <= r_FC)
                opti.subject_to(P_FC[t] - P_FC[t+1] <= r_FC)
                            
            # States                
            opti.subject_to(delta_ON_EL[t] + delta_OFF_EL[t] == 1)
            opti.subject_to(delta_ON_FC[t] + delta_OFF_FC[t] == 1)

            # System                
            #opti.subject_to(P_EL_sys[t] ==  P_EL[t] + P_EL_aux[t] )
            #opti.subject_to(P_FC_sys[t] ==  P_FC[t] - P_FC_aux[t] )
                                          
                            
            # Bilinear to linear


    # -----------------------------
    # Objective
    # -----------------------------

    obj = sum(alpha_EL * delta_ON_EL[t] + alpha_FC * delta_ON_FC[t] + c_battery * ca.fabs(P_batt_ch[t] - P_batt_disch[t]) + c_el[t]*P_grid[t]*t_s for t in range(N))
    opti.minimize(obj)

    # -----------------------------
    # Solve and get optimal values
    # -----------------------------

    sol = opti.solve()
    
    SOE_opt = sol.value(SOE)
    LOH_opt = sol.value(LOH)
    P_batt_disch_opt = sol.value(P_batt_disch)
    P_batt_ch_opt = sol.value(P_batt_ch)
    P_grid_opt = sol.value(P_grid)
    #P_FC_sys_opt = sol.value(P_FC_sys)
    #P_FC_aux_opt = sol.value(P_FC_aux)
    P_FC_opt = sol.value(P_FC)  
    delta_ON_FC_opt = sol.value(delta_ON_FC)
    #delta_STB_FC_opt = sol.value(delta_STB_FC)
    delta_OFF_FC_opt = sol.value(delta_OFF_FC)
    #P_EL_sys_opt = sol.value(P_EL_sys)
    #P_EL_aux_opt = sol.value(P_EL_aux)
    P_EL_opt = sol.value(P_EL)
    delta_ON_EL_opt = sol.value(delta_ON_EL)
    #delta_STB_EL_opt = sol.value(delta_STB_EL)
    delta_OFF_EL_opt = sol.value(delta_OFF_EL)

            
    obj_opt = round(sol.value(obj)/100,2)

    

    return SOE_opt, LOH_opt, P_batt_disch_opt, P_batt_ch_opt, P_grid_opt, delta_ON_EL_opt, delta_STB_EL_opt, delta_OFF_EL_opt, P_EL_sys_opt, P_EL_aux_opt, P_EL_opt, delta_ON_FC_opt, delta_STB_FC_opt, delta_OFF_FC_opt, P_FC_sys_opt, P_FC_aux_opt, P_FC_opt, obj_opt


In [ ]:
def optimizer1(T, t_s, c_el, pv, load, SOE_initial, LOH_initial): 
    # adapt inputs: T (horizon of simulation), ts (sampling time), SOE_battery, LOH_hess, pv, load, c_el (electricity prices)
    # states: SOE_battery, LOH_hess
    # inputs to FMU: P_battery, P_FC_sys (start with P_FC), P_El_sys (start with P_El), state_FC, state_EL

    # ITERATION 1: no auxiliaries, no thermal model, no transitions, no STB state
    # inputs variables in MPC: P_battery_disch, P_battery_ch, P_grid, P_FC, delta_ON_FC, delta_OFF_FC, P_EL, delta_ON_EL, delta_OFF_EL
    
    
    # Access global parameters
    global P_grid_max, P_battery_nom, eta_batt_ch, eta_batt_disch, SOE_min, SOE_max
    global LOH_min, LOH_max, C_max, eta_El, eta_FC, P_EL_min, P_EL_max, P_EL_STB, P_FC_min, P_FC_max, P_FC_STB, r_EL, r_FC
    global alpha_EL, alpha_FC, c_battery

    N = int(T // t_s)
    
    # Initialize
    import casadi as ca
    opti = ca.Opti()

    # -----------------------------
    # Variables and solver
    # -----------------------------

    SOE = opti.variable(1, N+1)  # state
    LOH = opti.variable(1, N+1)  # state
    P_batt_disch = opti.variable(1, N)  # input
    P_batt_ch = opti.variable(1, N)     # input
    P_grid_sale = opti.variable(1, N)        # input 
    P_grid_pchs = opti.variable(1, N)        # input 
    P_FC = opti.variable(1, N)          # input
    delta_ON_FC = opti.variable(1, N)   # binary input
    delta_STB_FC = opti.variable(1, N)   # binary input
    delta_OFF_FC = opti.variable(1, N)  # binary input
    P_EL = opti.variable(1, N)          # input
    delta_ON_EL = opti.variable(1, N)   # binary input
    delta_STB_EL = opti.variable(1, N)   # binary input
    delta_OFF_EL = opti.variable(1, N)  # binary input

    # delta_i are discrete variables (binary)
    discrete_var = [0]*(N+1) + [0]*(N+1) + [0]*N + [0]*N + [0]*N + [0]*N + [0]*N + [1]*N + [1]*N + [1]*N + [0]*N + [1]*N + [1]*N + [1]*N 

    # Solver
    opti.solver('bonmin', {'discrete': discrete_var, 'bonmin.tol': 1e-6, 'bonmin.print_level': 0, 'print_time': 0})

    # -----------------------------
    # Constraints
    # -----------------------------

    # Initial storage level
    opti.subject_to(SOE[0] == SOE_initial)
    opti.subject_to(LOH[0] == LOH_initial)

    # Constraints at every time step
    for t in range(N):

            # Bounds on grid
            opti.subject_to(pv[t] + P_grid_pchs[t] + P_batt_disch[t] + P_FC[t] - delta_STB_FC[t] * P_FC_STB == load[t] + P_grid_sale[t] + P_batt_ch[t] + P_EL[t] + delta_STB_EL[t] * P_EL_STB)
            opti.subject_to(P_grid_sale[t]>=0)
            opti.subject_to(P_grid_sale[t]<=P_grid_max)
            opti.subject_to(P_grid_pchs[t]>=0)
            opti.subject_to(P_grid_pchs[t]<=P_grid_max)

            # Bounds on battery and SOE
            opti.subject_to(P_batt_disch[t]>=0)
            opti.subject_to(P_batt_disch[t]<=0.9 * P_battery_nom)
            opti.subject_to(P_batt_ch[t]>=0)
            opti.subject_to(P_batt_ch[t]<=0.9 * P_battery_nom)   
        
            opti.subject_to(SOE[t+1] == SOE[t] + (P_batt_ch[t] * eta_batt_ch - P_batt_disch[t]/eta_batt_disch) * t_s)
            opti.subject_to(SOE[t] >= SOE_min)
            opti.subject_to(SOE[t] <= SOE_max)
            
            # Bounds on hydrogen storage 
            opti.subject_to(LOH[t+1] == LOH[t] + (P_EL[t] * eta_EL - P_FC[t]/eta_FC) * t_s /C_max)
            opti.subject_to(LOH[t] >= LOH_min)
            opti.subject_to(LOH[t] <= LOH_max)                

            # Constraints on electrolyzer
            # Bounds when ON                
            opti.subject_to(P_EL[t] >= delta_ON_EL[t] * P_EL_min)
            opti.subject_to(P_EL[t] <= delta_ON_EL[t] * P_EL_max)
        
            # Ramping constraints                
            if t < N-1:                
                opti.subject_to(P_EL[t+1] - P_EL[t] <= r_EL)
                opti.subject_to(P_EL[t] - P_EL[t+1] <= r_EL)
                            
            # Constraints on fuel cell
            # Bounds when ON
            opti.subject_to(P_FC[t] >= delta_ON_FC[t] * P_FC_min)
            opti.subject_to(P_FC[t] <= delta_ON_FC[t] * P_FC_max)
        
            # Ramping constraints                
            if t < N-1:                
                opti.subject_to(P_FC[t+1] - P_FC[t] <= r_FC)
                opti.subject_to(P_FC[t] - P_FC[t+1] <= r_FC)
                            
            # States                
            opti.subject_to(delta_ON_EL[t] + delta_STB_EL[t] + delta_OFF_EL[t] == 1)
            opti.subject_to(delta_ON_FC[t] + delta_STB_FC[t]  + delta_OFF_FC[t] == 1)


                            
            # Bilinear to linear


    # -----------------------------
    # Objective
    # -----------------------------

    obj = sum(alpha_EL/t_s * 3600 * delta_ON_EL[t] + alpha_FC/t_s * 3600 * delta_ON_FC[t] + c_battery * ca.fabs(P_batt_ch[t] - P_batt_disch[t]) + (c_el[t] * P_grid_pchs[t] - 1 * c_el[t] * P_grid_sale[t]) * t_s for t in range(N))
    opti.minimize(obj)
    
    # -----------------------------
    # Solve and get optimal values
    # -----------------------------

    sol = opti.solve()
    
    SOE_opt = sol.value(SOE)
    LOH_opt = sol.value(LOH)
    P_batt_disch_opt = sol.value(P_batt_disch)
    P_batt_ch_opt = sol.value(P_batt_ch)
    P_grid_sale_opt = sol.value(P_grid_sale)
    P_grid_pchs_opt = sol.value(P_grid_pchs)
    P_FC_opt = sol.value(P_FC)
    delta_ON_FC_opt = sol.value(delta_ON_FC)
    delta_STB_FC_opt = sol.value(delta_STB_FC)
    delta_OFF_FC_opt = sol.value(delta_OFF_FC)
    P_EL_opt = sol.value(P_EL)
    delta_ON_EL_opt = sol.value(delta_ON_EL)
    delta_STB_EL_opt = sol.value(delta_STB_EL)
    delta_OFF_EL_opt = sol.value(delta_OFF_EL)

    obj_opt = round(sol.value(obj) / 100, 2)

    return delta_STB_FC_opt, delta_STB_EL_opt, SOE_opt, LOH_opt, P_batt_disch_opt, P_batt_ch_opt, P_grid_sale_opt, P_grid_pchs_opt, delta_ON_EL_opt, delta_OFF_EL_opt, P_EL_opt, delta_ON_FC_opt, delta_OFF_FC_opt, P_FC_opt, obj_opt

In [1]:
# Define global simulation parameters
T = 3*60*60  # Time horizon length: horizon T = 3h to start
t_s = 60*60  # Time step size in seconds: ts = 1h -> for not losing too much on transitions for HESS, ideally ts should be smaller or equal to tau_CLD


# Define global parameters
R = 8.314
F = 1.602176634 *10**(-19)
M = 5000
m = 1
V_tn = 1.482
T_amb = 273.15 + 23.5
K=1.4

# Electrolyzer parameters
eta_EL = 0.6
eta_F_EL = 0.95 # Faradaic efficiency
P_EL_STB = 285.7
P_EL_nom = 5000
P_EL_max = 5000
P_EL_min = 0.1 * P_EL_max
r_EL = 4500 # 20 W/s or depeds on t_s
C_rep_EL = 0.27*1.55*10**(-3)  # replacement cost (euro/W) - to be multiplied by installed power in W P_EL_nom
N_H_EL = 40000     # number of hours
C_OM_EL = 0.002    # O&M cost (euro/h)
alpha_EL = 1000 * (C_rep_EL * P_EL_nom /N_H_EL + C_OM_EL) # (euro/Wh * W + euro/h)

# Electrolyzer auxiliaries parameters


# Fuel cell parameters
eta_FC = 0.4
eta_F_FC = 0.95 # Faradaic efficiency
P_FC_STB = 100
P_FC_nom = 5000
P_FC_max = 4300
P_FC_min = 300
r_FC = 5 * t_s # 5 W/s if t_s < 15min, otherwise = P_FC_max  
C_rep_FC = 0.27*1.55*10**(-3)  # replacement cost for the fuel cell (euro/W) - to be multiplied by installed power in W P_EL_nom
N_H_FC = 40000     # number of hours for the fuel cell (h)
C_OM_FC = 0.01    # O&M cost for the fuel cell (euro/h)
alpha_FC = 1000 * (C_rep_FC * P_FC_nom /N_H_FC + C_OM_FC) # (euro/Wh * W + euro/h)

# Fuel cell auxiliaries parameters


# Hydrogen storage parameters
LOH_min = 0.1 # (1)
LOH_max = 0.99 # (1)
C_max = 5158.1*10**6 # (J), for LHV = 119.96 MJ/kg

# Battery parameters
E_battery_nom = 5000 # (Wh)
C_battery_inv = 0.37  # investment cost for the battery (euro/Wh)
N_battery_cycles = 4000  # number of battery cycles
P_battery_nom = 5000 # (W)
c_battery = C_battery_inv * E_battery_nom / (2 * N_battery_cycles * P_battery_nom) # (euro/W)
eta_batt_ch = 0.98 # (1)
eta_batt_disch = 0.98 # (1)
SOE_min = 500 # (Wh)
SOE_max = 4500 # (Wh)

# Grid arameters
P_grid_max = 5000 # (W)

In [9]:
def optimizer(T, t_s, c_el, pv, load, SOE_initial, LOH_initial): 
    # adapt inputs: T (horizon of simulation), ts (sampling time), SOE_battery, LOH_hess, pv, load, c_el (electricity prices)
    # states: SOE_battery, LOH_hess
    # inputs to FMU: P_battery, P_FC_sys (start with P_FC), P_El_sys (start with P_El), state_FC, state_EL

    # ITERATION 1: no auxiliaries, no thermal model, no transitions, no STB state
    # inputs variables in MPC: P_battery_disch, P_battery_ch, P_grid, P_FC, delta_ON_FC, delta_OFF_FC, P_EL, delta_ON_EL, delta_OFF_EL
    # P_batt = P_batt, ch - P_batt,disch - we assume unitary efficiency 
    
    
    # Access global parameters
    global P_grid_max, P_battery_nom, eta_batt_ch, eta_batt_disch, SOE_min, SOE_max
    global LOH_min, LOH_max, C_max, eta_El, eta_FC, P_EL_min, P_EL_max, P_EL_STB, P_FC_min, P_FC_max, P_FC_STB, r_EL, r_FC
    global alpha_EL, alpha_FC, c_battery

    N = int(T // t_s)
    
    # Initialize
    import casadi as ca
    opti = ca.Opti()

    # -----------------------------
    # Variables and solver
    # -----------------------------

    SOE = opti.variable(1, N+1)  # state
    LOH = opti.variable(1, N+1)  # state
    P_batt = opti.variable(1, N)  # input
    P_grid_sale = opti.variable(1, N)        # input 
    P_grid_pchs = opti.variable(1, N)        # input 
    P_FC = opti.variable(1, N)          # input
    delta_ON_FC = opti.variable(1, N)   # binary input
    delta_OFF_FC = opti.variable(1, N)  # binary input
    P_EL = opti.variable(1, N)          # input
    delta_ON_EL = opti.variable(1, N)   # binary input
    delta_OFF_EL = opti.variable(1, N)  # binary input

    # delta_i are discrete variables (binary)
    discrete_var = [0]*(N+1) + [0]*(N+1) + [0]*N + [0]*N + [0]*N + [0]*N + [1]*N + [1]*N + [0]*N + [1]*N + [1]*N

    # Solver
    opti.solver('bonmin', {'discrete': discrete_var, 'bonmin.tol': 1e-4, 'bonmin.print_level': 0, 'print_time': 0})

    # -----------------------------
    # Constraints
    # -----------------------------

    # Initial storage level
    opti.subject_to(SOE[0] == SOE_initial)
    opti.subject_to(LOH[0] == LOH_initial)

    # Constraints at every time step
    for t in range(N):

            # Bounds on grid
            opti.subject_to(pv[t] + P_grid_pchs[t] + P_FC[t] == load[t] + P_grid_sale[t] + P_batt[t] + P_EL[t])
            opti.subject_to(P_grid_sale[t]>=0)
            opti.subject_to(P_grid_sale[t]<=P_grid_max)
            opti.subject_to(P_grid_pchs[t]>=0)
            opti.subject_to(P_grid_pchs[t]<=P_grid_max)

            # Bounds on battery and SOE
            opti.subject_to(P_batt[t]>=0)
            opti.subject_to(P_batt[t]<=0.9 * P_battery_nom)
        
            opti.subject_to(SOE[t+1] == SOE[t] + P_batt[t] * t_s/3600)
            opti.subject_to(SOE[t] >= SOE_min)
            opti.subject_to(SOE[t] <= SOE_max)
            
            # Bounds on hydrogen storage 
            opti.subject_to(LOH[t+1] == LOH[t] + (P_EL[t] * eta_EL - P_FC[t]/eta_FC) * t_s /C_max)
            opti.subject_to(LOH[t] >= LOH_min)
            opti.subject_to(LOH[t] <= LOH_max)                

            # Constraints on electrolyzer
            # Bounds when ON                
            opti.subject_to(P_EL[t] >= delta_ON_EL[t] * P_EL_min)
            opti.subject_to(P_EL[t] <= delta_ON_EL[t] * P_EL_max)
        
            # Ramping constraints                
            if t < N-1:                
                opti.subject_to(P_EL[t+1] - P_EL[t] <= r_EL)
                opti.subject_to(P_EL[t] - P_EL[t+1] <= r_EL)
                            
            # Constraints on fuel cell
            # Bounds when ON
            opti.subject_to(P_FC[t] >= delta_ON_FC[t] * P_FC_min)
            opti.subject_to(P_FC[t] <= delta_ON_FC[t] * P_FC_max)
            
            # Ramping constraints                
            if t < N-1:                
                opti.subject_to(P_FC[t+1] - P_FC[t] <= r_FC)
                opti.subject_to(P_FC[t] - P_FC[t+1] <= r_FC)
                            
            # States                
            opti.subject_to(delta_ON_EL[t] + delta_OFF_EL[t] == 1)
            opti.subject_to(delta_ON_FC[t] + delta_OFF_FC[t] == 1)


                            
            # Bilinear to linear


    # -----------------------------
    # Objective
    # -----------------------------

    obj = sum(alpha_EL * t_s/3600 * delta_ON_EL[t] + alpha_FC * t_s/3600 * delta_ON_FC[t] + c_battery * ca.fabs(P_batt[t]) + (c_el[t] * P_grid_pchs[t] - 0 * c_el[t] * P_grid_sale[t]) * t_s/3600 for t in range(N))
    opti.minimize(obj)
    
    # -----------------------------
    # Solve and get optimal values
    # -----------------------------

    sol = opti.solve()
    
    SOE_opt = sol.value(SOE)
    LOH_opt = sol.value(LOH)
    P_batt_opt = sol.value(P_batt)
    P_grid_sale_opt = sol.value(P_grid_sale)
    P_grid_pchs_opt = sol.value(P_grid_pchs)
    P_FC_opt = sol.value(P_FC)  
    delta_ON_FC_opt = sol.value(delta_ON_FC)
    delta_OFF_FC_opt = sol.value(delta_OFF_FC)
    P_EL_opt = sol.value(P_EL)
    delta_ON_EL_opt = sol.value(delta_ON_EL)
    delta_OFF_EL_opt = sol.value(delta_OFF_EL)

    obj_opt = round(sol.value(obj), 2)

    return SOE_opt, LOH_opt, P_batt_opt, P_grid_sale_opt, P_grid_pchs_opt, delta_ON_EL_opt, delta_OFF_EL_opt, P_EL_opt, delta_ON_FC_opt, delta_OFF_FC_opt, P_FC_opt, obj_opt
        

In [10]:
def ctrl_postprocessor(delta_ON_EL, delta_OFF_EL, delta_ON_FC, delta_OFF_FC, P_batt_ch, P_batt_disch, P_FC, P_EL):
    state_EL = []
    state_FC = []
    P_battery = []

    # Loop over each time step
    for i in range(len(delta_ON_EL)):
        # Determine the state of the electrolyzer
        if delta_ON_EL[i] == 1:
            state_EL.append(2)  # ON state
        elif delta_OFF_EL[i] == 1:
            state_EL.append(0)  # OFF state
        else:
            state_EL.append(1)  # Standby state

        # Determine the state of the fuel cell
        if delta_ON_FC[i] == 1:
            state_FC.append(2)  # ON state
        elif delta_OFF_FC[i] == 1:
            state_FC.append(0)  # OFF state
        else:
            state_FC.append(1)  # Standby state

        # Calculate the net battery power for each time step
        P_battery.append(P_batt_ch[i] - P_batt_disch[i])

    return np.array(P_battery), np.array(P_FC), np.array(P_EL), np.array(state_FC), np.array(state_EL)


In [12]:
import numpy as np
import matplotlib.pyplot as plt



c_el = np.array([0.3*10**(-3), 0.4*10**(-3), 0.28*10**(-3) ])  # Electricity prices in euro/Wh - around 0.3 euro/kWh
pv = np.array([1000, 3500, 4000])  # PV generation
load = np.array([4230, 4500, 4230])  # Load
SOE_initial = 2500  # Initial state of energy of the battery
LOH_initial = 0.5  # Initial level of hydrogen

# Call the optimizer function
SOE_opt, LOH_opt, P_batt_opt, P_grid_sale_opt, P_grid_pchs_opt, delta_ON_EL_opt, delta_OFF_EL_opt, P_EL_opt, delta_ON_FC_opt, delta_OFF_FC_opt, P_FC_opt, obj_opt = optimizer(T, t_s, c_el, pv, load, SOE_initial, LOH_initial)

#delta_STB_FC_opt, delta_STB_EL_opt, SOE_opt, LOH_opt, P_batt_disch_opt, P_batt_ch_opt, P_grid_sale_opt, P_grid_pchs_opt, delta_ON_EL_opt, delta_OFF_EL_opt, P_EL_opt, delta_ON_FC_opt, delta_OFF_FC_opt, P_FC_opt, obj_opt = optimizer1(T, t_s, c_el, pv, load, SOE_initial, LOH_initial)


NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT 1.4334239       12 0.139973
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT 1.4334        5 0.032565
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT 1.4334        5 0.0567
Cbc0012I Integer solution of 1.4334 found by DiveMIPFractional after 0 iterations and 0 nodes (0.11 seconds)
Cbc0001I Search completed - best objective 1.433400000084772, took 0 iterations and 0 nodes (0.11 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost


In [5]:
P_batt, P_FC, P_EL, state_FC, state_EL = ctrl_postprocessor(delta_ON_EL_opt, delta_OFF_EL_opt, delta_ON_FC_opt, delta_OFF_FC_opt, P_batt_ch_opt, P_batt_disch_opt, P_FC_opt, P_EL_opt)

In [ ]:
P_batt

In [ ]:
obj_opt

In [ ]:
# Time vector for plotting
time = np.arange(T/t_s + 1)  # Time vector with N+1 elements

plt.figure(figsize=(14, 10))

# Plot SOE
plt.subplot(3, 2, 1)
plt.plot(time[:-1], SOE_opt[:-1], marker='o')
plt.title('State of Energy (SOE) of the Battery')
plt.xlabel('Time [hours]')
plt.ylabel('SOE [Wh]')
plt.grid(True)

# Plot LOH
plt.subplot(3, 2, 2)
plt.plot(time[:-1], LOH_opt[:-1], marker='o')
plt.title('Level of Hydrogen (LOH)')
plt.xlabel('Time [hours]')
plt.ylabel('LOH [Wh]')
plt.grid(True)

# Plot P_grid
plt.subplot(3, 2, 3)
plt.plot(time[:-1], P_grid_pchs_opt, marker='o', label='Purchased from grid')
plt.plot(time[:-1], P_grid_sale_opt, marker='o', label='Sold to grid')
plt.title('Power Grid Usage')
plt.xlabel('Time [hours]')
plt.ylabel('Power [W]')
plt.legend()
plt.grid(True)

# Plot P_EL and P_FC
plt.subplot(3, 2, 4)
plt.plot(time[:-1], P_EL_opt, marker='o', label='P_EL')
plt.plot(time[:-1], P_FC_opt, marker='o', label='P_FC')
plt.title('Electrolyzer and Fuel Cell Power')
plt.xlabel('Time [hours]')
plt.ylabel('Power [W]')
plt.legend()
plt.grid(True)

# Plot all power contributions
plt.subplot(3, 2, (5, 6))
plt.plot(time[:-1], load, label='Load', marker='o')
plt.plot(time[:-1], pv, label='PV Generation', marker='o')
plt.plot(time[:-1], P_grid_pchs_opt - P_grid_sale_opt, label='Grid Power', marker='o')
plt.plot(time[:-1], P_batt_disch_opt -P_batt_ch_opt , label='Battery ', marker='o')
plt.plot(time[:-1], P_EL_opt, label='Electrolyzer Power (P_EL)', marker='o')
plt.plot(time[:-1], P_FC_opt, label='Fuel Cell Power (P_FC)', marker='o')
plt.title('Power Contributions')
plt.xlabel('Time [hours]')
plt.ylabel('Power [W]')
plt.legend()
plt.grid(True)



plt.tight_layout()
plt.show()


In [ ]:
time_control = np.arange(T/t_s)  # Time vector for control signals with N elements

# Plot delta_ON_FC and delta_OFF_FC
plt.subplot(4, 2, 1)
plt.step(time[:-1], delta_ON_FC_opt, label='delta_ON_FC', where='mid')
plt.step(time[:-1], delta_OFF_FC_opt, label='delta_OFF_FC', where='mid')
plt.title('Fuel Cell States')
plt.xlabel('Time [hours]')
plt.ylabel('State')
plt.legend()
plt.grid(True)

# Plot delta_ON_EL and delta_OFF_EL
plt.subplot(4, 2, 2)
plt.step(time[:-1], delta_ON_EL_opt, label='delta_ON_EL', where='mid')
plt.step(time[:-1], delta_OFF_EL_opt, label='delta_OFF_EL', where='mid')
plt.title('Electrolyzer States')
plt.xlabel('Time [hours]')
plt.ylabel('State')
plt.legend()
plt.grid(True)


plt.tight_layout()
plt.show()
